# 14 - Using External ASE Objects (Calculators/Optimizers)

### In this tutorial we will learn how to use external ASE calculators and optimizers within the architector framework.

**(A)** Using arbitrary methods to perform energy/relaxations.

**(B)** How to use example ASE calculator (MACE-MP-0) within Architector complex construction.

**(C)** How to use arbirtrary ASE optimizers within Architector.

To get started - make sure to install the MACE-MP-0 package and model:

See: https://github.com/ACEsuit/mace

In [ ]:
# Now, load in the correct functions and model
from architector import build_complex, view_structures, convert_io_molecule
from mace.calculators import mace_mp # Import Mace ASE calculator
calc = mace_mp(model="medium", dispersion=True,
               default_dtype="float32") # Instantiate the MACE calculator
from architector.io_calc import CalcExecutor # Calculation executor interface.
import numpy as np

Now, Let's generate a structure (UFF) for an arbitary SMILES string and distort the coordinates by a random distance:

In [ ]:
arb_smiles = 'c1ccccc1CON'
mol = convert_io_molecule(arb_smiles)
mol.ase_atoms.set_positions(mol.ase_atoms.get_positions() + np.random.random(mol.ase_atoms.get_positions().shape) / 2)
view_structures(mol)

Now, To relax this structure we can use the `CalcExeuctor` interface with MACE-MP-0:

In [ ]:
from ase.optimize import BFGS
out = CalcExecutor(mol,
                   relax=True, # Optimize the molecule
                   method='custom', # Use arbirtrary method.
                   calculator=calc,  # Set the calc to be MACE-MP-0
                   ase_opt_method=BFGS, # Set ASE optimizer
                   save_trajectories=True, # Save the trajectories
                   debug=True) # Show the printout to verify it is using the methods asked for.

In [ ]:
# Now we can view the relaxation trajectory in its entirety
print('Relaxed')
view_structures(out.mol)
print('Trajectory')
view_structures(out.trajectory, trajectory=True, interval=600)

In [ ]:
# Also, the energy and initial energies, and any errors are stored in the output object
for key, val in out.__dict__.items():
    if (key != 'parameters') and (key not in out.__dict__['parameters']):
        print(key, ':', val)

## (B) How to use example ASE calculator (MACE-MP-0) within Architector complex construction.
Here, we can directly pluging calculators to the full architector search:

In [ ]:
inputDict = {'core':{'metal':'Fe','coreCN':6}, # Specify metal coordination number (CN)
            'ligands':['water']*6, # Specify what is filling the coordination environment
            'parameters':{'calculator':calc, # Assign ASE calculator 
                          'assemble_method':'UFF', # Use UFF for assembly
                          'full_method':'custom'}}  # Give 'custom' as a flag to use alternative calculator
fe_h2o_6_out = build_complex(inputDict); # Now just build using the dictionary!
view_structures(fe_h2o_6_out)

Note that Architector will assign the unpaired electrons to the ASE atoms object in the atoms
`initial magnetic moments` and the charges in the `initial charges` placeholders.

Implementation of calculators may differ on whether this works or not.

## (C) How to use arbirtrary ASE optimizers within Architector.

Here, we can directly use other ASE optimizers as well, the default is LBFGSLineSearch. So Let's USE BFGS.

In [ ]:
inputDict = {'core':{'metal':'Fe','coreCN':1}, # Specify metal coordination number (CN)
            'ligands':['water'], # Specify what is filling the coordination environment
            'parameters':{'ase_opt_method':BFGS, # Directly pass the optimizer object
                          'debug':True # Use debug to show that it is using BFGS instead of LBFGSLineSearch
                          }} 
fe_h2o_out = build_complex(inputDict); # Now just build using the dictionary!

In [ ]:
view_structures(fe_h2o_out)

### Summary: In this tutorial we learned how to use external ASE calculators and optimizers within the architector framework. Including:

**(A)** Using arbitrary methods to perform energy/relaxations.

**(B)** How to use example ASE calculator (MACE-MP-0) within Architector complex construction.

**(C)** How to use arbirtrary ASE optimizers within Architector.